<a href="https://colab.research.google.com/github/ivancloudvm/Examen_NLP/blob/main/Analisis_sentimientos_apli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis de sentimientos Apli- [Large Movie Review Dataset v1.0](http://ai.stanford.edu/~amaas/data/sentiment/)

## Iván Vázquez Martínez

Se usó un script para extraer los datos y crear 2 archivos separados de train y test con todas las reviews:

#Script para extraer los datos descargados de http://ai.stanford.edu/~amaas/data/sentiment/ y crear 2 archivos con todos los datos en train y test\

#!/bin/bash

\# unzip y unpack el archivo tar:\
gunzip -c aclImdb_v1.tar.gz | tar xopf -

cd aclImdb

\#creamos directorio donde estarán los archivos de train y test\
mkdir movie_data

\# Creamos los archivos donde estarán todos los datos de train y test, el orden es importante ya que las primeras 12.5k filas tendrán label positiva y las siguientes negativas\
\# full_train.txt, full_test.txt:\
for split in train test;\
do

  for sentiment in pos neg;\
  do 
    
    for file in $split/$sentiment/*; 
    do
              cat $file >> movie_data/full_${split}.txt; 
              echo >> movie_data/full_${split}.txt; 

	    
    done;
  done;\
done;

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Importamos librerias</h1></center>


In [17]:
import numpy as np
import pandas as pd
import os
import re
import glob
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Cargamos los datos</h1></center>


In [18]:
# train_test = [r"\train",r"\test"]
# pos_neg = [r"\pos",r"\neg"]

# all_files = []
# for i in range(len(train_test)):
#     for j in range(len(pos_neg)):
#         rel_path = train_test[i] + pos_neg[j]
#         os.chdir(r"C:\Users\sephc\Python\aclImdb"+ rel_path)
#         extension = 'txt'
#         files = [i for i in glob.glob('*.{}'.format(extension))]
#         all_files = all_files + files
# len(all_files)
# # Los primeros 25000 archivos son train de los cuales los primeros 12500 son pos (1) y los siguientes son neg (0)
# # luego los siguientes 25000 archivos son test de los cuales los primeros 12500 son pos (1) y los siguientes son neg (0).

# full_trainp = all_files[:25000]
# full_testp = all_files[25000:]

# len(full_trainp)

In [19]:
reviews_train = []
for line in open('/content/drive/MyDrive/full_train.txt', 'r', encoding="utf8"):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('/content/drive/MyDrive/full_test.txt', 'r',encoding="utf8"):
    
    reviews_test.append(line.strip())

In [20]:
reviews_train[0]

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Preprocesamiento</h1></center>


Creamos la función preprocess_reviews() para reemplazar caracteres innecesarios:

In [21]:
#patterns
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

#funcion
def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    return reviews

#reviews limpias:
reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [22]:
reviews_train_clean[0]

'bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my  years in the teaching profession lead me to believe that bromwell highs satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled  at  high a classic line inspector im here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isnt'

In [23]:
len(reviews_train_clean)

25000

In [24]:
len(reviews_test_clean)

25000

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Vectorizamos</h1></center>


In [25]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Modelo y Accuracy</h1></center>


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

target = [1 if i < 12500 else 0 for i in range(25000)]

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c, max_iter = 1000)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy con C=0.01: 0.86608
Accuracy con C=0.05: 0.88064
Accuracy con C=0.25: 0.88064
Accuracy con C=0.5: 0.87808
Accuracy con C=1: 0.87584


Con esto concluimos que el coeficiente de regularización óptimo es de C = 0.05:

In [27]:
modelo_final = LogisticRegression(C=0.05)
modelo_final.fit(X, target)
print ("Accuracy: %s"
       % accuracy_score(target, modelo_final.predict(X_test)))

Accuracy: 0.88144


In [28]:
modelo_final.coef_[0]

array([-0.00823153,  0.04268005,  0.04294587, ...,  0.00025657,
       -0.02673009, -0.00672057])

In [29]:
modelo_final.classes_

array([0, 1])

In [30]:
modelo_final.intercept_

array([0.28491093])

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">GridSearchCV</h1></center>

In [31]:
# from sklearn import svm
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression

# model_params = {
#     'svm': {
#         'model': svm.SVC(gamma='auto'),
#         'params' : {
#             'C': [1,5,10],
#             'kernel': ['rbf','linear']
#         }  
#     },
#     'random_forest': {
#         'model': RandomForestClassifier(),
#         'params' : {
#             'n_estimators': [1,5,10]
#         }
#     }
# }

In [32]:
# from sklearn.model_selection import GridSearchCV
# scores = []

#for model_name, mp in model_params.items():
#     clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
#     clf.fit(X, target)
#     scores.append({
#         'model': model_name,
#         'best_score': clf.best_score_,
#         'best_params': clf.best_params_
#     })
    
#df_results = pd.DataFrame(scores,columns=['model','best_score','best_params'])
#df_results

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Curva ROC</h1></center>


In [33]:
# from sklearn.metrics import roc_curve, roc_auc_score
# y_pred_proba= modelo_final.predict_proba(X_val)
# fpr, tpr, thresholds = roc_curve(y_val,y_pred_proba[:,1])

# plt.plot(fpr,tpr)
# plt.plot([0,1],[0,1],linestyle='--')
# plt.xlim([0.0,1.0])
# plt.ylim([0.0,1.0])
# plt.xlabel('1-specificity')
# plt.ylabel('sensitivity')
# plt.show()

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Mejores 5 y peores palabras</h1></center>


In [34]:
feature_to_coef = {word: coef for word, coef in zip(cv.get_feature_names(), modelo_final.coef_[0])}

for best_positive in sorted(feature_to_coef.items(), key=lambda x: x[1], reverse=True)[:5]:
    print (best_positive)

('excellent', 0.9283544357387583)
('perfect', 0.7944277750867349)
('great', 0.6745552873944645)
('amazing', 0.6164834551994242)
('superb', 0.6055919705160007)


In [35]:
for best_negative in sorted(feature_to_coef.items(), key=lambda x: x[1])[:5]:
    print (best_negative)

('worst', -1.367989768332016)
('waste', -1.1688808937647217)
('awful', -1.0273337505628393)
('poorly', -0.8748022407884607)
('boring', -0.8591221171268896)


### Quitamos las Stopwords

In [37]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(no_stop_words_train)
X = cv.transform(no_stop_words_train)
X_test = cv.transform(no_stop_words_test)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c, max_iter = 1000)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Accuracy con C=0.01: 0.87456
Accuracy con C=0.05: 0.88192
Accuracy con C=0.25: 0.87648
Accuracy con C=0.5: 0.87376
Accuracy con C=1: 0.87104


Con C = 0.05 se obtiene la mejor accuracy

## Radicalización (stemming)

In [38]:
def get_stemmed_text(corpus):
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

stemmed_reviews_train = get_stemmed_text(reviews_train_clean)
stemmed_reviews_test = get_stemmed_text(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(stemmed_reviews_train)
X = cv.transform(stemmed_reviews_train)
X_test = cv.transform(stemmed_reviews_test)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
final_stemmed = LogisticRegression(C=0.05, max_iter = 5000)
final_stemmed.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final_stemmed.predict(X_test)))

Accuracy con C=0.01: 0.86976
Accuracy con C=0.05: 0.876


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy con C=0.25: 0.87648


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy con C=0.5: 0.8736


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy con C=1: 0.87088
Accuracy: 0.8772


## Lematización

In [40]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def get_lemmatized_text(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

lemmatized_reviews_train = get_lemmatized_text(reviews_train_clean)
lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(lemmatized_reviews_train)
X = cv.transform(lemmatized_reviews_train)
X_test = cv.transform(lemmatized_reviews_test)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c,max_iter = 2000)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
final_lemmatized = LogisticRegression(C=0.25)
final_lemmatized.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final_lemmatized.predict(X_test)))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Accuracy con C=0.01: 0.87264
Accuracy con C=0.05: 0.8824
Accuracy con C=0.25: 0.88544
Accuracy con C=0.5: 0.88432
Accuracy con C=1: 0.88176
Accuracy: 0.87364


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## n-grams

In [41]:
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c, max_iter = 5000)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
final_ngram = LogisticRegression(C=0.5, max_iter = 5000)
final_ngram.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final_ngram.predict(X_test)))

Accuracy con C=0.01: 0.88112
Accuracy con C=0.05: 0.88976
Accuracy con C=0.25: 0.89136
Accuracy con C=0.5: 0.89296
Accuracy con C=1: 0.89296
Accuracy: 0.89872


### TF-IDF

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(reviews_train_clean)
X = tfidf_vectorizer.transform(reviews_train_clean)
X_test = tfidf_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
final_tfidf = LogisticRegression(C=1, max_iter = 2000)
final_tfidf.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final_tfidf.predict(X_test)))

Accuracy con C=0.01: 0.78528
Accuracy con C=0.05: 0.81872
Accuracy con C=0.25: 0.86336
Accuracy con C=0.5: 0.8768
Accuracy con C=1: 0.88544
Accuracy: 0.88248


### Linear SVC

In [43]:
from sklearn.svm import LinearSVC

ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c, max_iter = 2000)
    svm.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))
    

final_svm_ngram = LinearSVC(C=0.01,max_iter = 2000)
final_svm_ngram.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final_svm_ngram.predict(X_test)))

Accuracy con C=0.01: 0.88976
Accuracy con C=0.05: 0.88544
Accuracy con C=0.25: 0.88384
Accuracy con C=0.5: 0.88384
Accuracy con C=1: 0.88336
Accuracy: 0.8976


### LinearSVC sin Stopwords

In [44]:
stop_words = ['in', 'of', 'at', 'a', 'the']
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words=stop_words)
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.001, 0.005, 0.01, 0.05, 0.1]:
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))
    
final = LinearSVC(C=0.01)
final.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final.predict(X_test)))

Accuracy con C=0.001: 0.88832
Accuracy con C=0.005: 0.89232
Accuracy con C=0.01: 0.89264
Accuracy con C=0.05: 0.89232
Accuracy con C=0.1: 0.89216
Accuracy: 0.9


### XGBoost

In [45]:
# from xgboost import XGBRegressor
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split

# stop_words = ['in', 'of', 'at', 'a', 'the']
# ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words=stop_words)
# ngram_vectorizer.fit(reviews_train_clean)
# X = ngram_vectorizer.transform(reviews_train_clean)
# X_test = ngram_vectorizer.transform(reviews_test_clean)

# X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

# estimators = [100,300,500,800,1000]

# xgboost = XGBRegressor(n_estimators =100 , learning_rate = 0.05)
# xgboost.fit(X_train, y_train,
#             early_stopping_rounds = 5,
#             eval_set =[(X_val,y_val)],
#             verbose = False)
# print ("Accuracy for n=%s: %s" 
#         % (100, accuracy_score(y_val, xgboost.predict(X_val))))

In [46]:
X_train.get_shape()

(18750, 5398364)

In [47]:
len(y_train)

18750

In [48]:
#!pip install keras

In [49]:
#!pip install tensorflow

In [58]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

In [61]:
type(X_train)

scipy.sparse.csr.csr_matrix

In [57]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_val = np.asarray(X_val)
y_val = np.asarray(y_val)

type(X_train)

numpy.ndarray

### Neural Network

In [56]:
from keras import models
from keras import layers


model = models.Sequential()
# Input - Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(18750, )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

# Compilamos el modelo
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])




results = model.fit(
 X_train, y_train,
 epochs= 2,
 batch_size = 500,
 validation_data = (X_val, y_val))

print("Test-Accuracy:", np.mean(results.history["val_acc"]))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 50)                937550    
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 51        
Total params: 942,701
Trainable params: 942,701
Non-trainable params: 0
________________________________________________

ValueError: ignored

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Creación de Dataframe</h1></center>


In [ ]:
reviews_test_series = pd.Series(reviews_test_clean)
reviews_train_series = pd.Series(reviews_train_clean)

target_series = pd.Series(target)

frame_test = { 'Reviews': reviews_test_series, 'Sentiment': target_series }
frame_train = { 'Reviews': reviews_train_series, 'Sentiment': target_series }

df_test = pd.DataFrame(frame_test)
df_train = pd.DataFrame(frame_train)

df_train[12498:12502]

In [ ]:
target_series[12498:12502]

In [ ]:
df_train.describe()